In [1]:
%run VECTOR_DB_BUILD.ipynb     #this is were my function was stored

In [2]:
from transformers import pipeline, AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
import accelerate
import torch

In [3]:
READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    READER_MODEL_NAME, quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

<|system|>
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}</s>
<|assistant|>



In [23]:
questions = []
for i in range(5):
    print('Ask your question')
    q = str(input())
    questions.append(q)

Ask your question


 What is an LLM?


Ask your question


 What is the difference between a sql data base and a vector data base?


Ask your question


 how to book a movie ticket?


Ask your question


 what is BSP domain controllers?


Ask your question


 what are the MSP high level requirements?


In [24]:
for questions in questions:
    retrieved_docs = main_retriever.get_relevant_documents(questions)
    retrieved_docs_text = [
        doc.page_content for doc in retrieved_docs
    ]  # we only need the text of the documents
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)]
    )
    final_prompt = RAG_PROMPT_TEMPLATE.format(
        question=questions, context=context
    )
        
    # Redact an answer
    answer = READER_LLM(final_prompt)[0]["generated_text"]
    print(answer)
    print('\n')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

An LLM (Large Language Model) is a type of artificial intelligence algorithm that can generate human-like text based on a given prompt or input. It uses deep learning techniques to process large amounts of text data and can perform tasks such as language translation, summarization, and question answering with high accuracy. In some cases, LLMs are also used for text generation and creative writing purposes. (Source: Document 0)

In more detail, LLMs are trained on vast datasets of text, typically billions of words, using unsupervised learning methods. They learn to predict the next word in a sequence based on the context of the previous words, allowing them to generate coherent and meaningful text. Some popular examples of LLMs include GPT-3 (Generative Pretrained Transformer 3), BERT (Bidirectional Encoder Representations from Transformers), and RoBERT (Robustly Optimized BERT Pretraining Approach). (Sources: Documents 1-3)




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

A SQL (Structured Query Language) database stores and manages structured data in tables with defined schemas. It allows for querying, updating, and deleting data using SQL statements. In contrast, a vector database, such as those offered by LlamaIndex, stores pre-computed vector representations of text data for fast retrieval and similarity search. Instead of searching through text, it performs calculations on numerical vectors that represent the meaning or semantic content of the text. This allows for more efficient and accurate search results based on similarity rather than exact matches. While SQL databases are useful for storing and managing structured data, vector databases excel at handling large volumes of unstructured text data for tasks like information retrieval and recommendation systems. (Answer provided based on the given context.)




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Based on the provided context, it does not offer information on how to book a movie ticket. The extracts only provide URLs that allow third-party ticketing providers to access specific subscriber records. To answer your question, you may need to provide additional context or details about the location and theater where you wish to watch the movie. Without this information, it is impossible to provide a comprehensive answer.

However, if you have identified a specific theater or chain that offers online booking, you can visit their website or mobile app and follow the steps to select the desired movie, date, time, and seats. Alternatively, you can use popular ticketing platforms like Bookmyshow (https://in.bookmyshow.com/), Paytm (https://movie.paytm.com/), or Insider (https://insider.in/) to find and book tickets for various theaters across India. These websites and apps offer a simple and convenient way to reserve your seat and receive e-tickets via email or SMS.

In summary, to book 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In the provided context, BSP domain controllers refer to the network infrastructure managed by the BSP (Broadband Service Provider) that hosts the high-level requirements for Tier 1 in Calix Cloud. This means that Calix Cloud runs within this network infrastructure managed by the BSP. The exact functionality and responsibilities of these domain controllers are not explicitly defined in the given context, but they appear to play a role in managing the network environment for Calix Cloud at the Tier 1 level. Document 1, 2, and 3 each provide a summary of the high-level requirements for BSP domain controllers at Tier 1, suggesting that there may be some variation or customization required depending on the specific deployment scenario. However, it's unclear whether all Tier 2 deployments require BSP domain controllers or if they can opt to use only the Access Controller component instead. Further details about the BSP domain controllers would likely be found in the full documentation or im

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Based on Documents 1-3 provided, the MSP (managed service provider) high level requirements can be summarized as follows:

1. Scale and hardware guidelines are similar to those for standalone instances (SMx).
2. Detailed requirements may vary depending on specific MSP scenarios, but high-level goals include providing examples, rephrasing queries, and stating the overall objective at a broad level.

Unfortunately, without access to the full content of these documents, it's impossible to provide more detailed or specific requirements beyond what has been stated. However, based on this information, it's clear that the MSP requirements share some similarities with those for standalone instances, while also having unique needs related to managing services for multiple clients.


